In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
path = 'data/healthcare-dataset-stroke-data.csv'

dataframe = pd.read_csv(path)
dataframe.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


### FP-Growth algoritm 

I'm going to use thia algorithm to find frequent patterns from the data set that can help to identify patternas that characterize the stroke. 
This algoritm work in the next way:
1. List the items of the data set
2. Scan the whole data set, count each item and remove infrequent items.
3. Group the items in order of their frequency from bigger to smaller , and create a tree where each path represents the connection between the items and common combinations.


This Algorithm is more efficient then Apriori algorithm, due to it computation cost. Apriori algorithm each time create the new group of of k items that connect and based on this groups is scan the data set one more time to create thegroups of k+1 items.(Candidate generation and than filter them by support rate).

FP-Growth Algorithm works with categorical values, and can't handle numerical ,in our data set we have the numerical values like: age, glucose level ,hypertension, bmi, stroke . 

For the items like age,glucose level and bmi i'll use the binning  and discretization techniques to convert them to categorical values.

For hypertension ,stroke and heart deciase I'll use discritization.

### PreProcessing Data

In [3]:
cp_dataframe = dataframe.copy()

In [5]:
cp_dataframe = cp_dataframe.drop(axis=1,columns="id")

In [7]:
bins = [0, 2, 18, 40, 60, 82]
labels = ['newborn', 'child', 'young adult', 'middle age', 'senior']

cp_dataframe['age_group'] = pd.cut(cp_dataframe['age'],bins = bins,labels = labels,right=True)

In [8]:
cp_dataframe

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,age_group
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1,senior
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1,senior
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1,senior
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1,middle age
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1,senior
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0,senior
5106,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0,senior
5107,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0,young adult
5108,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0,middle age


In [17]:
print(max(cp_dataframe['bmi']),min(cp_dataframe['bmi']),max(cp_dataframe['avg_glucose_level']),min(cp_dataframe['avg_glucose_level']))

94.8 17.618581952117864 271.74 55.12


In [10]:
bmi_mean_by_age = cp_dataframe.groupby('age_group')['bmi'].mean().to_dict()

/var/folders/28/pmh07v693dn2c3l0jdz5zvlw0000gn/T/ipykernel_9604/754360870.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bmi_mean_by_age = cp_dataframe.groupby('age_group')['bmi'].mean().to_dict()


Create the bins for glucose level and bmi. For glucose levele i have bins of the size 20, and for bmi 10.

In [19]:
for key,val in bmi_mean_by_age.items():
    mask = (cp_dataframe['age_group'] == key) & (cp_dataframe['bmi'].isna())
    cp_dataframe.loc[mask,'bmi'] = val
    
cp_dataframe['bmi'] = cp_dataframe['bmi'].round(2)

#discretization and smoothing for bmi
# Define bin edges from 10 to 100 with a step of 10
bin_edges_bmi = np.arange(10, 100 + 10, 10)  # [10, 20, 30, ..., 100]

bin_edges_glucose = np.arange(55,275,20)
# Create BMI bins
cp_dataframe['bmi_bin'] = pd.cut(cp_dataframe['bmi'], bins=bin_edges_bmi, include_lowest=True)
#Create Glucose Bins 
cp_dataframe['avg_glucose_bin']= pd.cut(cp_dataframe['avg_glucose_level'], bins=bin_edges_glucose, include_lowest=True)




In [20]:
cp_dataframe

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,age_group,bmi_bin,avg_glucose_bin
0,Male,67.0,0,1,Yes,Private,Urban,228.69,33.91,formerly smoked,1,senior,"(30.0, 40.0]","(215.0, 235.0]"
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,25.68,never smoked,1,senior,"(20.0, 30.0]","(195.0, 215.0]"
2,Male,80.0,0,1,Yes,Private,Rural,105.92,33.91,never smoked,1,senior,"(30.0, 40.0]","(95.0, 115.0]"
3,Female,49.0,0,0,Yes,Private,Urban,171.23,33.91,smokes,1,middle age,"(30.0, 40.0]","(155.0, 175.0]"
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,25.68,never smoked,1,senior,"(20.0, 30.0]","(155.0, 175.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5105,Female,80.0,1,0,Yes,Private,Urban,83.75,25.68,never smoked,0,senior,"(20.0, 30.0]","(75.0, 95.0]"
5106,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,33.91,never smoked,0,senior,"(30.0, 40.0]","(115.0, 135.0]"
5107,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,33.91,never smoked,0,young adult,"(30.0, 40.0]","(75.0, 95.0]"
5108,Male,51.0,0,0,Yes,Private,Rural,166.29,25.68,formerly smoked,0,middle age,"(20.0, 30.0]","(155.0, 175.0]"
